In [16]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [17]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.12.0+cu113  Device: cuda


In [18]:
BATCH_SIZE = 32
EPOCHS = 10

In [19]:
train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                  train = True,
                                  download = True,
                                  transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,
                                transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

Files already downloaded and verified


In [20]:
import torchvision.models as models
model = models.resnet34(pretrained = False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
modes = model.cuda()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)   
criterion = nn.CrossEntropyLoss()

print(model)


In [22]:
def train(model, train_loader, optimizer,log_interval):
  model.train()
  for batch_idx, (image, label) in enumerate(train_loader):
    image = image.to(DEVICE)
    label = label.to(DEVICE)
    optimizer.zero_grad()
    output = model(image)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()

In [23]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for image, label in test_loader:
      image = image.to(DEVICE)
      label = label.to(DEVICE)
      output = model(image)
      test_loss += criterion(output, label).item()
      prediction = output.max(1, keepdim = True)[1]
      correct += prediction.eq(label.view_as(prediction)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [24]:
for Epoch in range(1, EPOCHS+1):
  train(model, train_loader, optimizer,log_interval = 200)
  test_loss, test_accuracy = evaluate(model, test_loader)
  print("[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n".format(Epoch,test_loss, test_accuracy))

[EPOCH: 1], 	Test Loss: 0.0389, 	Test Accuracy: 56.40 %

[EPOCH: 2], 	Test Loss: 0.0349, 	Test Accuracy: 62.92 %

[EPOCH: 3], 	Test Loss: 0.0270, 	Test Accuracy: 70.45 %

[EPOCH: 4], 	Test Loss: 0.0249, 	Test Accuracy: 72.78 %

[EPOCH: 5], 	Test Loss: 0.0223, 	Test Accuracy: 75.28 %

[EPOCH: 6], 	Test Loss: 0.0229, 	Test Accuracy: 74.72 %

[EPOCH: 7], 	Test Loss: 0.0223, 	Test Accuracy: 75.71 %

[EPOCH: 8], 	Test Loss: 0.0197, 	Test Accuracy: 78.85 %

[EPOCH: 9], 	Test Loss: 0.0204, 	Test Accuracy: 77.81 %

[EPOCH: 10], 	Test Loss: 0.0207, 	Test Accuracy: 78.12 %



현재 변화된 부분은 'Resnet_CNN_pytorch' 코드에서 모델 구현 부분만 있음

1.기존 구현 코드에서 class ResNet이 아닌, torchvision.models의 resnet34를 model에 집어넣어 학습시킴. 기존보다 정확도가 떨어지는 현상이 보이고, 이를 보완할 방법으로 Transfer Learning을 찾음.

2.학습 진행 속도가 기존 class ResNet보다 많이 느림. 왜그런지 궁금함
